*U-net to segment endothelial cells in specular microscopy images*

Include libraries

In [0]:
import numpy as np
import tensorflow as tf
import random as rn
import glob
import re

Configure and start a tensorflow session

In [0]:
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)

Import keras specific classes

In [0]:
from keras.models import *
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, concatenate
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as keras
import argparse
from keras.utils import plot_model
import keras
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


Set the session in keras

In [0]:
K.set_session(sess)
print (keras.__version__)

2.2.4


Mount google drive to access training and testing images and labels

In [0]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Define the loss function to train the neural network

We will use a weighted form the cross entropy loss function

(Less commonly seen classes will have heigher weights)

Cell border pixels are weighted nearly 4 times more as compared to other pixels

In [0]:
def weighted_binary_crossentropy(y_true, y_pred):

	y_true_f = K.flatten(y_true)
	y_pred_f = K.flatten(y_pred)

	binary_crossentropy = K.binary_crossentropy(y_true_f, y_pred_f)
	weighted_vector = y_true_f * 0.21 + (1. - y_true_f) * 0.79
	weighted_binary_crossentropy_loss = weighted_vector * binary_crossentropy

	return K.mean(weighted_binary_crossentropy_loss)

Helper function to sort images in the right order

For example, 1.bmp, 2.bmp, 3.bmp instead of 1.bmp, 10.bmp, 11.bmp etc.

In [0]:
def natural_keys(text):

    # Assumes that the path is of the form /content/.../1.jpg
    c = re.split('(/\d+)', text)
    print(c)
    return int(c[1].split('/')[1])

Define the U-Net class, will contain routines to read/write data, train/test networks

Object to this class will be instantiated in __main__

Specify location to the training and testing data in data_dir variable

__load_data__ reads the training images, labels and testing images from the google drive

__get_unet__ specifies the network architecture for U-net

__train_and_test__ trains and tests the neural network

In [0]:
class myUnet(object):

    def __init__(self):

        self.data_dir = '/content/drive/My Drive/Colab Notebooks/'
    
    def load_data(self):

        # Augment the training images and labels, because 34 training images is a small number of images for deep learning methods
        datagen_args = dict(rotation_range=0.2,
                            width_shift_range=0.05,
                            height_shift_range=0.05,
                            shear_range= 0.05,
                            zoom_range=0.05,
                            horizontal_flip=True,
                            fill_mode='nearest',
                            rescale=1./255)
        
        image_datagen = ImageDataGenerator(**datagen_args)
        labels_datagen = ImageDataGenerator(**datagen_args)

        
        # Load training images and labels from the respective directories
        image_generator = image_datagen.flow_from_directory(self.data_dir + 'data/train/image/', color_mode='grayscale', class_mode=None, seed=1, batch_size=1, target_size=(480, 320))
        labels_generator = labels_datagen.flow_from_directory(self.data_dir + 'data/train/label/', color_mode='grayscale', class_mode=None, seed=1, batch_size=1, target_size=(480, 320))
        
        # Find all test images from the data folder
        self.test_imgs_list = glob.glob(self.data_dir + 'test/image/*.bmp')

        # Sort so that the list is 1.jpg, 2.jpg etc. and not 1.jpg, 11.jpg etc.
        self.test_imgs_list.sort(key=natural_keys)

        # Read test image files and load them into a numpy array
        imgs_test_stack = np.zeros((len(self.test_imgs_list), 480, 320))
        for i in np.arange(len(self.test_imgs_list)):
            imgs_test_stack[i,:,:] = img_to_array(load_img(self.test_imgs_list[i]))[:,:,1]/255
        
        print('%d EC test images found' %(len(self.test_imgs_list)))
        
        #adds a dimension of 1 at the end
        imgs_test_stack = np.expand_dims(imgs_test_stack, axis=-1)

        return image_generator, labels_generator, imgs_test_stack
      
    def get_unet(self):

        # Neural network architecture
        # Based on the U-Net described here: https://arxiv.org/abs/1505.04597
        
        # Input layer shape
        inputs = Input((480, 320, 1))        

        # First Encoding Block
        # Convolutional layers of 128 filters each, learning 3x3 kernel
        # ReLU activation. Kernel weights initialized as 2/sqrt(fan_in), 
        # where fan_in is the number of units feeding into this network.
        # Maximum pooling "pools" a 2x2 area and picks the maximum value
        # Gives some form of spatial invariance
        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        # Second Encoding block
        # Same as previous, but with more filters learnt (256 vs. 128)
        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
        conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

        # Third Encoding Block
        # Dropout (connections are randomly dropped in the network)
        # Proportion of connections dropped is 0.5
        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
        conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
        drop4 = Dropout(0.5)(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

        # Fourth Encoding Block
        conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
        conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
        drop5 = Dropout(0.5)(conv5)

        # First Decoding (Upsampling) Block
        # Transposed convolutions with kernel size (2,2)
        # Subsequent concatenation from lower encoding block outputs
        # Two convolutional layers
        up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
        merge6 = concatenate([drop4, up6], axis=3)
        conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
        conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

        # Second Decoding Block
        up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
        merge7 = concatenate([conv3, up7], axis=3)
        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
        conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

        # Third Decoding Block
        up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
        merge8 = concatenate([conv2, up8], axis=3)
        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
        conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
        conv8 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
        conv9 = Conv2D(1, 1, activation = 'sigmoid')(conv8)

        # Define the inputs and outputs to the network
        model = Model(inputs = inputs, outputs = conv9)

        # Use the weighted cross entropy loss function as described above
        # Adam optimizer is used with a learning rate of 1e-4
        model.compile(optimizer = Adam(lr = 1e-4), loss = weighted_binary_crossentropy,  metrics = ['accuracy'])

        model.summary()

        return model

    def train_and_test(self):

            print('-' * 30)
            print('Training on EC microscopy images')
            print('-' * 30)

            print('Loading training data (EC cells in microscopy images)')
            imgs_train, imgs_train_labels, imgs_test = self.load_data()
            print('Loaded training data (EC cells in microscopy images) \n')

            # Get the neural network architecture
            print('Loading network architecture')
            model = self.get_unet()
            print('Loaded network architecture \n')

            # Create a checkpoint to save the network weights to a file. Loss on the validation set will be monitored
            model_checkpoint = ModelCheckpoint(self.data_dir + 'unet_train.hdf5', monitor='loss',verbose=1, save_best_only=True)

            # Fit the model to the training data
            print('Fitting model to train dataset')
            model.fit_generator(zip(imgs_train, imgs_train_labels), steps_per_epoch=32, epochs=5, verbose=1, shuffle=True, callbacks=[model_checkpoint])

            print('-' * 30)
            print('Test on unseen EC microscopy images')
            print('-' * 30)

            # Load the network weights
            print('Loading network weights from unet_train.hdf5 file')
            model.load_weights(self.data_dir + 'unet_train.hdf5')
            print('Loaded weights from the pre-trained network \n')

            # Predict on the test images
            imgs_test_predictions = model.predict(imgs_test, batch_size=1, verbose=1)
            print('Predicted on test EC images')

            # Save predictions to the results folder
            print('Saving predictions on test images to results folder in the current directory')
            for i in np.arange(imgs_test_predictions.shape[0]):
              
              img_test_prediction = array_to_img(imgs_test_predictions[i,:,:])
              filename_start_index = self.test_imgs_list[i].rfind('/')

              img_test_prediction.save(self.data_dir + 'results/%s' %(self.test_imgs_list[i][filename_start_index+1:]))

if __name__ == '__main__':

    # Make an object of myUnet class
    myunet = myUnet()

    # Train and test the U-Net network as needed
    myunet.train_and_test()
    

------------------------------
Training on EC microscopy images
------------------------------
Loading training data (EC cells in microscopy images)
Found 32 images belonging to 1 classes.
Found 32 images belonging to 1 classes.
['/content/drive/My Drive/Colab Notebooks/train/images/all', '/14', '.bmp']
['/content/drive/My Drive/Colab Notebooks/train/images/all', '/23', '.bmp']
['/content/drive/My Drive/Colab Notebooks/train/images/all', '/22', '.bmp']
['/content/drive/My Drive/Colab Notebooks/train/images/all', '/20', '.bmp']
['/content/drive/My Drive/Colab Notebooks/train/images/all', '/21', '.bmp']
['/content/drive/My Drive/Colab Notebooks/train/images/all', '/15', '.bmp']
['/content/drive/My Drive/Colab Notebooks/train/images/all', '/16', '.bmp']
['/content/drive/My Drive/Colab Notebooks/train/images/all', '/24', '.bmp']
['/content/drive/My Drive/Colab Notebooks/train/images/all', '/18', '.bmp']
['/content/drive/My Drive/Colab Notebooks/train/images/all', '/28', '.bmp']
['/content/